## Packages

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

import pandas as pd
import time
import re

## Get Html 

In [7]:
def getter(url):
    browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(1)
    browser.refresh()
    time.sleep(1)
    browser.refresh()
    
    elem = browser.find_element_by_tag_name("body")
    
    no_of_pagedowns = 30
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
        no_of_pagedowns-=1
    html = browser.page_source
    return html
url4 = 'https://www.hotels.com/search.do?resolved-location=CITY%3A659171%3AUNKNOWN%3AUNKNOWN&f-star-rating=4&destination-id=659171&q-destination=Yogyakarta,%20Special%20Region%20of%20Yogyakarta,%20Indonesia&q-check-in=2020-08-01&q-check-out=2020-08-02&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&sort-order=DISTANCE_FROM_LANDMARK&sort-lid=1667134'
url5 = 'https://www.hotels.com/search.do?resolved-location=CITY%3A659171%3AUNKNOWN%3AUNKNOWN&f-star-rating=5&destination-id=659171&q-destination=Yogyakarta,%20Special%20Region%20of%20Yogyakarta,%20Indonesia&q-check-in=2020-08-01&q-check-out=2020-08-02&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&sort-order=DISTANCE_FROM_LANDMARK&sort-lid=1667134'
url = 'https://www.hotels.com/search.do?resolved-location=CITY%3A659171%3AUNKNOWN%3AUNKNOWN&f-star-rating=5,4&destination-id=659171&q-destination=Yogyakarta,%20Special%20Region%20of%20Yogyakarta,%20Indonesia&q-check-in=2020-08-01&q-check-out=2020-08-02&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&sort-order=DISTANCE_FROM_LANDMARK&sort-lid=1667134'
html = getter(url4)

## PseudoCode

In [4]:
"""
PseudoCode 1
1. untuk setiap card
2. coba dapatkan data
3. jika data ada tambahkan ke list
4. jika kosong tambahkan '0 Reviews'
PROBLEM : Menyimpan card kedalam list

PseudoCode 2 
1. list semua data 
2. gabungkan list 
3. ubah ke dataframe
PROBLEM : item kosong tidak berada di index yang benar

PseudoCode 3
1. untuk setiap card
2. dapatkan nama hotel append listItem
3. dapatkan address append listItem
4. dapatkan rate append listItem
5. end for
6. gabungkan list
""" 

"\nPseudoCode 1\n1. untuk setiap card\n2. coba dapatkan data\n3. jika data ada tambahkan ke list\n4. jika kosong tambahkan '0 Reviews'\nPROBLEM : Menyimpan card kedalam list\n\nPseudoCode 2 \n1. list semua data \n2. gabungkan list \n3. ubah ke dataframe\nPROBLEM : item kosong tidak berada di index yang benar\n\nPseudoCode 3\n1. untuk setiap card\n2. dapatkan nama hotel append listItem\n3. dapatkan address append listItem\n4. dapatkan rate append listItem\n5. end for\n6. gabungkan list\n"

## Scrap Data

In [8]:
soup = BeautifulSoup(html,'html.parser')
itemList = []
dfList  = []
def getcity():
    city = item.find('ul',class_="property-landmarks").get_text()
    city = city.split(')', 1)
    city = city[1]
    city = city.split(' to', 1)
    city = city[0]
    return city   
def getairport():
    airport = item.find('ul',class_="property-landmarks").get_text()
    airport = airport.split(' to', 1)
    airport = airport[0]
    return airport

In [9]:
card = soup.find_all('section',class_="hotel-wrap")
naRe = 0
naPr = 0
for item in card:
    try:
        rate = item.find('strong',class_="guest-reviews-badge").get_text()
        name = item.find('a',class_="property-name-link").get_text()
        address = item.find('span',class_="address").get_text()
        review = item.find('span',class_="small-view").get_text()
        review = re.sub("\D", "", review)
        airport = getairport()
        city = getcity()
    except AttributeError:
        rate = ''
        review = '' 
        naRe = naRe + 1
    try:
        price = item.find('ins').get_text()
    except:
        try:
            price = ''
            div = item.find('a',class_="price-link")
            price = div.find('strong').get_text()
        except:
            naPr = naPr + 1
            
    itemList = [name,address,rate,review,airport,city,price]
    dfList.append(itemList)

print('There is ', naRe ,' Hotels have No Reviews')
print('There is ', naPr ,' Hotels Fullbooked')

There is  0  Hotels have No Reviews
There is  3  Hotels Fullbooked


In [10]:
df = pd.DataFrame (dfList)
df.columns = ['Hotel_Name','Address','Rate','Reviews','Airport','City','Starting_Price']
df.head()

,Hotel_Name,Address,Rate,Reviews,Airport,City,Starting_Price
0,Platinum Adisucipto Hotel & Conference,"Jl. Solo No.28, Yogyakarta, Depok, 55282, Indo...",Fabulous 8.6,41,0.2 km,8.0 km,Rp791.736
1,Sheraton Mustika Yogyakarta Resort and Spa,"JL Laksda Adisucipto KM 8.7, Depok, Yogyakarta...",Very good 8.4,102,1.1 km,7.1 km,Rp600.000
2,Harper Mangkubumi by ASTON,"Jl. Mangkubumi No. 52, Yogyakarta, Yogyakarta,...",Fabulous 8.8,85,7.7 km,1.2 km,
3,GQ Hotel Yogyakarta,"Jl Adisucipto No 48, Depok, Yogyakarta, 55281,...",Very good 8.2,10,1.3 km,6.8 km,Rp2.228.979
4,INNSiDE by Melia Yogyakarta,"JL. Ring Road Utara, Maguwoharjo, Depok, Yogya...",Very good 8.0,41,2.2 km,7.9 km,Rp520.000


## Cleaning Data

In [96]:
### Remove Punctuation
# df['Hotel Name'] = df['Hotel Name'].str.replace('[^\w\s\1]','')

# df['Rate'] = df['Rate'].astype('str')
# df['Rate'] = df['Rate'].str.replace('[','')

## Airport Distance
#df['Airport'] = df['Airport'].astype('str')
# df['Airport'] = df['Airport'].split(' to', 1)
# df['Airport'] = df.Airport[0]
df.dtypes

Hotel_Name        object
Address           object
Rate              object
Reviews           object
Airport           object
City              object
Starting_Price    object
dtype: object

## Regex 

In [97]:
## txt = '6.2 km to Yogyakarta (JOG-Adisucipto Intl.)4.7 km to City centre'
# txt = txt.split(' to', 1)
# txt = txt[0]
# txt

# div = soup.find('a',class_="price-link")
# price = div.find('strong').get_text()
# price

city = soup.find('ul',class_="property-landmarks").get_text()
city = city.split(')', 1)
city = city[1]
city = city.split(' to', 1)
city = city[0]
city

'8.0 km'

## Export to CSV

In [12]:
# Export
df.to_csv(r'Hotel-List-4Star.csv', index = False)